In [1]:
from tensorflow.keras.preprocessing import image
import tensorflow as tf
import numpy as np
import os
from dotenv import load_dotenv
from find_image import FindImage
import botocore
import json

2023-10-05 20:21:49.073351: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-05 20:21:49.073381: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-05 20:21:49.073412: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-05 20:21:49.079190: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-05 20:21:49.993535: W tensorflow/compiler/

In [2]:
tf.sysconfig.get_build_info()

OrderedDict([('cpu_compiler', '/usr/lib/llvm-16/bin/clang'),
             ('cuda_compute_capabilities',
              ['sm_35', 'sm_50', 'sm_60', 'sm_70', 'sm_75', 'compute_80']),
             ('cuda_version', '11.8'),
             ('cudnn_version', '8'),
             ('is_cuda_build', True),
             ('is_rocm_build', False),
             ('is_tensorrt_build', True)])

In [5]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
tf.debugging.set_log_device_placement(True)

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[0], 'GPU')

In [2]:
load_dotenv()

True

In [3]:
# Loading model
loaded_model = tf.keras.models.load_model(os.getenv('MODEL_PATH'))

2023-10-04 22:57:43.868240: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-04 22:57:43.868892: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


KeyboardInterrupt: 

In [ ]:
with open('class_names.json', 'r') as file:
    class_names_generated = json.load(file)

In [ ]:
def match_pet():
  # Carregando e redimensionando a imagem
  img = image.load_img(os.getenv('PATH_DOWNLOADED_IMG'), target_size=(224, 224))
  img_array = image.img_to_array(img)
  img_array = np.expand_dims(img_array, axis=0)
  img_array /= 255.0  # Normalização

  # Fazendo a previsão
  prediction = loaded_model.predict(img_array)
  
  # Obtendo a classe prevista
  predicted_class_index = np.argmax(prediction)

  # Obtendo o nome da classe
  predicted_class = class_names_generated[f"{predicted_class_index}"]


  # Imprimindo a classe prevista
  print(predicted_class)

In [ ]:
s3 = FindImage(flag=True, bucket_name=os.getenv("BUCKET_S3"), resource_name="s3")

try:
    print(os.getenv('PATH_DOWNLOADED_IMG'))
    #s3.download(os.getenv('FILE_KEY'), os.getenv('PATH_DOWNLOADED_IMG'))
    print(f"Arquivo baixado com sucesso.")
    match_pet()
except botocore.exceptions.NoCredentialsError:
    print("Credenciais não encontradas ou inválidas.")
except botocore.exceptions.ClientError as e:
    if e.response["Error"]["Code"] == "404":
        print("O arquivo não foi encontrado no bucket.")
    else:
        print("Erro desconhecido ao baixar o arquivo:", e)

/home/patrick/SPTECH/TCC/grupo11-4cco-tcc/src/inteligencia_artificial/imgs_test/dog1.jpg
Arquivo baixado com sucesso.
1/1 [==============================] - 0s 24ms/step
dog1
